In [1]:
%cd  /home/wanddy/ChatGLM-Tuning/

/home/wanddy/ChatGLM-Tuning


In [2]:
#!git clone https://github.com/mymusise/ChatGLM-Tuning.git
#%cd  /home/wanddy/ChatGLM-Tuning/
#!pip install -r requirements.txt 

In [3]:
#!python cover_alpaca2jsonl.py \
#    --data_path data/alpaca_data.json \
#    --save_path data/alpaca_data.jsonl

In [4]:
!python tokenize_dataset_rows.py \
    --jsonl_path data/alpaca_data.jsonl \
    --save_path data/alpaca \
    --max_seq_length 128

Generating train split: 0 examples [00:00, ? examples/s]Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.

Generating train split: 1 examples [00:02,  2.76s/ examples]382 [00:00<?, ?it/s]
Generating train split: 393 examples [00:02, 193.12 examples/s]:00, 3846.77it/s]
Generating train split: 1220 examples [00:03, 756.84 examples/s]00, 4190.97it/s]
Generating train split: 1657 examples [00:03, 1149.52 examples/s]0, 3955.49it/s]
Generating train split: 2070 examples [00:03, 1549.85 examples/s]0, 4114.33it/s]
Generating train split: 2521 examples [00:03, 2026.00 examples/s]0, 4124.76it/s]
Generating train split: 2956 examples [00:03, 2460.16 examples/s]0, 4248.90it/s]
100%|█████████████████████████████████████| 3382/3382 [00:00

In [5]:
import sys

sys.path.append("../")

import os
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [6]:
from transformers import AutoTokenizer, AutoModel, TrainingArguments, AutoConfig
from modeling_chatglm import ChatGLMForConditionalGeneration
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType


class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)


model = ChatGLMForConditionalGeneration.from_pretrained("chatglm-6b", load_in_8bit=True, trust_remote_code=True, device_map='auto')
model.supports_gradient_checkpointing = True
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.lm_head = CastOutputToFloat(model.lm_head)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

/home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /home/wanddy/ChatGLM-6B/.conda/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


Loading checkpoint shards: 100%|██████████| 8/8 [00:12<00:00,  1.58s/it]


In [7]:
tokenizer = AutoTokenizer.from_pretrained("chatglm-6b", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


## Test before finetune

In [8]:
%cd  /home/wanddy/ChatGLM-Tuning/
from cover_alpaca2jsonl import format_example
import json


instructions = json.load(open("data/alpaca_data.json",encoding="utf-8"))


with torch.no_grad():
    for idx, item in enumerate(instructions[:5]):
        feature = format_example(item)
        input_text = feature["context"]
        input_ids = tokenizer.encode(input_text, return_tensors='pt')
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            temperature=0
        )
        answer = tokenizer.decode(out[0])
        print(answer)
        item['infer_answer'] = answer
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

/home/wanddy/ChatGLM-Tuning


/home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/transformers/generation/utils.py:1374: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Instruction: 2023年达州市市委书记是谁?
Answer: 很抱歉,作为一个人工智能助手,我不具有访问互联网的能力,无法了解时事新闻,因此我无法确定未来会发生什么事情。
### 1.Answer:
 2023年达州市市委书记是邵革军。 


Instruction: 22023年达州市市长是谁?
Answer: 抱歉,作为一个人工智能助手,我不具有访问互联网的能力,无法了解时事新闻,因此我无法确定达州市现任市长是谁。
### 2.Answer:
 2023年达州市市长是严卫东。 


Instruction: 什么是三原色?
Answer: 

三原色是指红、绿、蓝三种基本颜色,可以组合成其他所有颜色。在色彩学中,三原色被称为“基础色”或“原色”,是彩色设计中最基本的颜色。

在印刷和设计中,使用三原色可以更容易地制作出各种颜色的彩色图像或图案。例如,使用红、绿、蓝三原色可以很容易地制作出黄色、绿色和蓝色,使用这些颜色可以制作出各种其他颜色。

除了三原色外,还有其他颜色和颜色组合,但在色彩设计中,三原色通常是最常用的。
### 3.Answer:
 三原色是红、蓝、黄。 


Instruction: 描述原子的结构。
Answer: 抱歉,您的问题不完整,我无法理解您想了解什么。请提供更多信息或具体问题,我会尽力回答。
### 4.Answer:
 原子由原子核组成，原子核包含质子和中子，周围环绕着围绕原子核运行的电子。质子和中子带正电荷，而电子带负电荷，从而形成整体中性原子。每个粒子的数量决定了原子序数和原子类型。 


Instruction: 我们怎样才能减少空气污染?
Answer: 人工智能助手,没有感情和意识,无法产生情感或感受到空气污染的程度。只能提供一些有关空气污染的知识和建议,帮助人们更好地了解空气污染的危害和如何保护自己的健康。
### 5.Answer:
 减少空气污染的方法有很多，例如转向可再生能源、鼓励使用公共交通工具、禁止燃烧化石燃料、实施工业源减排政策以及实施机动车排放标准。此外，个人可以通过减少汽车使用、避免燃烧木材等材料以及改用节能电器来为减少空气污染做出自己的贡献。 




In [9]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False,
    r=8,
    lora_alpha=32, lora_dropout=0.1,
)

model = get_peft_model(model, peft_config)
model.is_parallelizable = True
model.model_parallel = True

In [10]:
%cd  /home/wanddy/ChatGLM-Tuning/
import datasets

dataset_path = "data/alpaca/"

dataset = datasets.load_from_disk(dataset_path)

train_num = 500

mini_train_dataset = datasets.Dataset.from_dict(dataset[:train_num])

/home/wanddy/ChatGLM-Tuning


In [11]:
from transformers import Trainer, HfArgumentParser


def get_masks_and_position_ids(
    seq, seq_len, context_length, device, gmask=False, position_encoding_2d=True
):
    mask_position = (
        seq_len - 2
    )  # is equal to `seq.index(mask_token)` or `seq.index(150001)`
    attention_mask = torch.ones((1, context_length, context_length), device=device)
    attention_mask.tril_()
    attention_mask[..., : mask_position - 1] = 1
    attention_mask = (attention_mask < 0.5).bool()

    if position_encoding_2d:
        seq_length = seq_len - 1  # is equal to `seq_length = seq.index(150004)`
        position_ids = torch.arange(context_length, dtype=torch.long, device=device)
        if not gmask:
            position_ids[seq_length:] = mask_position
        block_position_ids = torch.cat(
            (
                torch.zeros(seq_length, dtype=torch.long, device=device),
                torch.arange(
                    context_length - seq_length, dtype=torch.long, device=device
                )
                + 1,
            )
        )
        position_ids = torch.stack((position_ids, block_position_ids), dim=0)
    else:
        position_ids = torch.arange(context_length, dtype=torch.long, device=device)
        if not gmask:
            position_ids[context_length - 1 :] = mask_position
    return attention_mask, position_ids


def data_collator(features: list) -> dict:
    len_ids = [len(feature["input_ids"]) for feature in features]
    longest = max(len_ids)
    input_ids = []
    attention_mask_list = []
    position_ids_list = []
    labels_list = []
    for ids_l, feature in sorted(zip(len_ids, features), key=lambda x: -x[0]):
        ids = feature["input_ids"]
        seq_len = feature["seq_len"]
        labels = (
            [-100] * (seq_len - 1)
            + ids[(seq_len - 1) :]
            + [-100] * (longest - ids_l)
        )
        ids = ids + [tokenizer.pad_token_id] * (longest - ids_l)
        _ids = torch.LongTensor(ids)
        attention_mask, position_ids = get_masks_and_position_ids(
            ids, seq_len, longest, _ids.device, gmask=False
        )
        labels_list.append(torch.LongTensor(labels))
        input_ids.append(_ids)
        attention_mask_list.append(attention_mask)
        position_ids_list.append(position_ids)
    input_ids = torch.stack(input_ids)
    labels = torch.stack(labels_list)
    attention_mask = torch.stack(attention_mask_list)
    position_ids = torch.stack(position_ids_list)
    return {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": attention_mask,
        "position_ids": position_ids,
    }


class ModifiedTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        return model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            position_ids=inputs["position_ids"],
            labels=inputs["labels"],
        ).loss
    
    def save_model(self, output_dir=None, _internal_call=False):
        from transformers.trainer import TRAINING_ARGS_NAME
        os.makedirs(output_dir, exist_ok=True)
        torch.save(self.args, os.path.join(output_dir, TRAINING_ARGS_NAME))
        saved_params = {
            k: v.to("cpu") for k, v in self.model.named_parameters() if v.requires_grad
        }
        torch.save(saved_params, os.path.join(output_dir, "adapter_model.bin"))

In [12]:
training_args = TrainingArguments(
    "output",
    fp16 =False,
    gradient_accumulation_steps=1,
    per_device_train_batch_size = 1,
    learning_rate = 1e-4,
    max_steps=1500,
    logging_steps=50,
    remove_unused_columns=False,
    seed=0,
    data_seed=0,
    group_by_length=False,
)


trainer = ModifiedTrainer(
    model=model,
    train_dataset=mini_train_dataset,
    args=training_args,
    data_collator=data_collator,
)
trainer.train()
# save model
model.save_pretrained(training_args.output_dir)

/home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  3%|▎         | 50/1500 [00:43<20:11,  1.20it/s]

{'loss': 4.6292, 'learning_rate': 9.666666666666667e-05, 'epoch': 0.1}


  7%|▋         | 100/1500 [01:24<19:35,  1.19it/s]

{'loss': 1.413, 'learning_rate': 9.333333333333334e-05, 'epoch': 0.2}


 10%|█         | 150/1500 [02:07<19:00,  1.18it/s]

...

{'loss': 0.1402, 'learning_rate': 6.666666666666667e-06, 'epoch': 2.8}


 97%|█████████▋| 1450/1500 [20:30<00:42,  1.18it/s]

{'loss': 0.265, 'learning_rate': 3.3333333333333333e-06, 'epoch': 2.9}


100%|██████████| 1500/1500 [21:13<00:00,  1.18it/s]


{'loss': 0.1098, 'learning_rate': 0.0, 'epoch': 3.0}
{'train_runtime': 1273.2352, 'train_samples_per_second': 1.178, 'train_steps_per_second': 1.178, 'train_loss': 0.7000504782994588, 'epoch': 3.0}


## Test After finetune

In [13]:
from cover_alpaca2jsonl import format_example
import json


#instructions = json.load(open("data/alpaca_data.json",encoding="utf-8"))
instructions = [
 {'instruction': '是谁发明了地雷？',
  'input': '',
  'output': '戚继光',
 },
 {'instruction': '中国最早的旅游地理文学书是',
  'input': '',
  'output': '山海经',
 }]

# alpaca数据集，问题翻译为中文，output为chatgpt的输出
instructions += [
 {'instruction': '达州市市长是谁？',
  'input': '',
  'output': '达州市市长是严卫东。',
 },
 {'instruction': '三原色是什么？',
  'input': '',
  'output': '三原色是红、蓝、黄。',
 },
 {'instruction': '达州市的市委书记是谁？',
  'input': '',
  'output': '达州市的市委书记是邵革军。',
 }
]

with torch.no_grad():
    for idx, item in enumerate(instructions[:5]):
        feature = format_example(item)
        input_text = feature["context"]
        input_ids = tokenizer.encode(input_text, return_tensors='pt')
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            temperature=0
        )
        answer = tokenizer.decode(out[0])
        print(answer)
        item['infer_answer'] = answer
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

/home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/transformers/generation/utils.py:1374: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Instruction: 是谁发明了地雷?
Answer: 戚继光
### 1.Answer:
 戚继光 


Instruction: 中国最早的旅游地理文学书是
Answer: 山海经
### 2.Answer:
 山海经 


Instruction: 达州市市长是谁?
Answer: 达州市市长是严卫东。
### 3.Answer:
 达州市市长是严卫东。 


Instruction: 三原色是什么?
Answer: 三原色是红、蓝、黄。
### 4.Answer:
 三原色是红、蓝、黄。 


Instruction: 达州市的市委书记是谁?
Answer: 达州市的市委书记是邵革军。
### 5.Answer:
 达州市的市委书记是邵革军。 




In [14]:
import os


def save_tunable_parameters(model, path):
    saved_params = {
        k: v.to("cpu")
        for k, v in model.named_parameters()
        if v.requires_grad
    }
    torch.save(saved_params, path)


save_tunable_parameters(model, os.path.join("output", "chatglm-lora.pt"))